# Feature Extraction

In [6]:
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import smbus
import time
import os
import math

In [17]:
# Define a class for the accelerometer readings
class accelerometer():
    def __init__(self):# ADXL345 address, 0x53(83)
        self.bus = smbus.SMBus(1)
    
    def get_x(self):
        # ADXL345 address, 0x53(83)
        # Read data back from 0x32(50), 2 bytes
        # X-Axis LSB, X-Axis MSB
        data_0 = bus.read_byte_data(0x53, 0x32)
        data_1 = bus.read_byte_data(0x53, 0x33)

        # Convert the data to 10-bits
        x_accl = ((data_1 & 0x03) * 256) + data_0
        if x_accl > 511:
            x_accl -= 1024
        return x_accl
    
    def get_y(self):
        # ADXL345 address, 0x53(83)
        # Read data back from 0x34(52), 2 bytes
        # Y-Axis LSB, Y-Axis MSB
        data_0 = bus.read_byte_data(0x53, 0x34)
        data_1 = bus.read_byte_data(0x53, 0x35)

        # Convert the data to 10-bits
        y_accl = ((data_1 & 0x03) * 256) + data_0
        if y_accl  > 511:
            y_accl -= 1024
        return y_accl
    
    def get_z(self):
        # ADXL345 address, 0x53(83)
        # Read data back from 0x36(54), 2 bytes
        # Z-Axis LSB, Z-Axis MSB
        data_0 = bus.read_byte_data(0x53, 0x36)
        data_1 = bus.read_byte_data(0x53, 0x37)

        # Convert the data to 10-bits
        z_accl = ((data_1 & 0x03) * 256) + data_0
        if z_accl > 511:
            z_accl -= 1024
        return z_accl
    
    def capture(self):
            # Select bandwidth rate register, 0x2C(44)
        #		0x0A(10)	Normal mode, Output data rate = 100 Hz
        self.bus.write_byte_data(0x53, 0x2C, 0x0A)
        # ADXL345 address, 0x53(83)
        # Select power control register, 0x2D(45)
        #		0x08(08)	Auto Sleep disable
        self.bus.write_byte_data(0x53, 0x2D, 0x08)
        # ADXL345 address, 0x53(83)
        # Select data format register, 0x31(49)
        #		0x08(08)	Self test disabled, 4-wire interface
        #					Full resolution, Range = +/-2g
        self.bus.write_byte_data(0x53, 0x31, 0x08)

    def get_read(self):
        self.capture()
        return (self.get_x(),self.get_y(),self.get_z())
    

In [19]:
acc = accelerometer()
 
for a in range(100):
    vals = acc.get_read()
    print(vals)
    #print("X {} - Y {} - Z {}".format(vals))
    time.sleep(0.2)
    os.system("clear")

(-180, -111, -93)
(-148, -63, -109)
(-308, 14, -107)
(-182, 43, -81)
(-230, 14, -107)
(-220, 54, -143)
(-200, 40, -145)
(-209, 10, -141)
(-204, 16, -138)
(-202, 27, -136)
(-196, 20, -132)
(-198, 29, -132)
(-204, 22, -133)
(-215, 13, -141)
(-208, 22, -142)
(-204, 5, -147)
(-172, 15, -198)
(-122, -8, -228)
(-36, 37, -265)
(57, 81, -244)
(-21, 58, -247)
(7, 73, -225)
(-67, 69, -210)
(-144, 77, -198)
(-127, 90, -175)
(-148, 95, -179)
(-155, 93, -173)
(-132, 114, -162)
(-162, 121, -178)
(-118, 125, -197)
(-78, 124, -202)
(-75, 103, -241)
(-52, 121, -211)
(-58, 148, -200)
(-139, 175, -215)
(-87, 188, -161)
(-99, 205, -141)
(-84, 208, -153)
(-93, 123, -222)
(-63, 128, -224)
(-98, 69, -226)
(-145, 63, -196)
(-175, 36, -165)
(-453, 164, -150)
(-247, -1, -47)
(-254, -58, -38)
(-282, -59, -318)
(-202, -107, -64)
(-217, -99, -56)
(-214, -88, -57)
(-217, -87, -55)
(-214, -90, -52)
(-216, -91, -52)
(-215, -90, -49)
(-215, -92, -48)
(-216, -93, -48)
(-217, -90, -47)
(-217, -91, -45)
(-215, -92, -42)


In [1]:
import tmp102

# Create figure for plotting
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
xs = []
ys = []

# Initialize communication with TMP102
tmp102.init()

# This function is called periodically from FuncAnimation
def animate(i, xs, ys):

    # Read temperature (Celsius) from TMP102
    temp_c = round(tmp102.read_temp(), 2)

    # Add x and y to lists
    xs.append(dt.datetime.now().strftime('%H:%M:%S.%f'))
    ys.append(temp_c)

    # Limit x and y lists to 20 items
    xs = xs[-20:]
    ys = ys[-20:]

    # Draw x and y lists
    ax.clear()
    ax.plot(xs, ys)

    # Format plot
    plt.xticks(rotation=45, ha='right')
    plt.subplots_adjust(bottom=0.30)
    plt.title('TMP102 Temperature over Time')
    plt.ylabel('Temperature (deg C)')

# Set up plot to call animate() function periodically
ani = animation.FuncAnimation(fig, animate, fargs=(xs, ys), interval=1000)
plt.show()


ImportError: No module named 'tmp102'